In [6]:
!pip install --upgrade dagshub pandas numpy

In [3]:
import dagshub
dagshub.init(repo_owner='nmach22', repo_name='house-prices-ml', mlflow=True)


Accessing as nmach22

Initialized MLflow to track repo "nmach22/house-prices-ml"

Repository nmach22/house-prices-ml initialized!

In [5]:
data_path =  './data/house-prices-advanced-regression-techniques'

In [7]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk(data_path):
    for filename in filenames:
        print(os.path.join(dirname, filename))


./data/house-prices-advanced-regression-techniques/test.csv
./data/house-prices-advanced-regression-techniques/data_description.txt
./data/house-prices-advanced-regression-techniques/train.csv
./data/house-prices-advanced-regression-techniques/sample_submission.csv


In [87]:
import mlflow
logged_model = 'runs:/904a3fb9a1554234a2ef0bbf38de61a6/pipeline_model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

In [88]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: pipeline_model
  flavor: mlflow.sklearn
  run_id: 904a3fb9a1554234a2ef0bbf38de61a6

In [89]:
data_types = loaded_model.input_example.dtypes.to_dict()

In [90]:
test_df = pd.read_csv(os.path.join(data_path, 'test.csv'), dtype=data_types, engine='python', dtype_backend='numpy_nullable')
test_df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal


In [91]:
y_pred_scaled = loaded_model.predict(test_df)
y_pred = np.exp(y_pred_scaled)

In [92]:
y_pred

array([126392.58094981, 154080.89445146, 182750.61689387, ...,
       151792.41349844, 107497.81173608, 231444.64039838], shape=(1459,))

### Generate Submission file

In [94]:
submission = pd.DataFrame({
    'Id': test_df['Id'],
    'SalePrice': y_pred.squeeze()
})

# Save to CSV
submission.to_csv('Submission_RandomForest(random_seed=40, y_scaler=log(price) ).csv', index=False)

In [95]:
submission

,Id,SalePrice
0,1461,126392.580950
1,1462,154080.894451
2,1463,182750.616894
3,1464,181128.862451
4,1465,197438.411938
...,...,...
1454,2915,83921.376329
1455,2916,82599.503525
1456,2917,151792.413498
1457,2918,107497.811736
